In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models
from compact.knowledge_distillation import KnowledgeDistillation

In [2]:
conf = Config()
load_models = MLP_Models()
teacher_model = load_models.mlp_4
teacher_model.load()
student_model = load_models.result_light_mlp_1
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="result"
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/mlp_4.pth!


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([32, 513]) torch.Size([32])


In [5]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 74.09%, Train loss: 0.0163, Val loss: 2.2720
Epoch: 2/10, Accuracy: 87.56%, Train loss: 0.0085, Val loss: 0.2352
Epoch: 3/10, Accuracy: 76.02%, Train loss: 0.0075, Val loss: 0.3917
Epoch: 4/10, Accuracy: 84.87%, Train loss: 0.0070, Val loss: 0.2638
Epoch: 5/10, Accuracy: 85.31%, Train loss: 0.0066, Val loss: 0.2221
Epoch: 6/10, Accuracy: 85.14%, Train loss: 0.0063, Val loss: 0.2480
Epoch: 7/10, Accuracy: 90.18%, Train loss: 0.0062, Val loss: 0.1736
Epoch: 8/10, Accuracy: 86.37%, Train loss: 0.0060, Val loss: 0.2490
Epoch: 9/10, Accuracy: 90.32%, Train loss: 0.0059, Val loss: 0.1713
Epoch: 10/10, Accuracy: 90.29%, Train loss: 0.0058, Val loss: 0.1708


In [6]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 91.51%


In [7]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 90.22%


In [8]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/result_light_mlp_1.pth
